Graph-Maker 2000

In [ ]:
###### Script to generate morphometric boxplots for all limpet species (shell & keyhole) ######



#### Imports ####

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch
import os



#### Configuration ####

# File paths
file_path_lc_lg_ls = 'shell_march2025_measuremetns_standardized_measurements.csv'
file_path_fv = 'hole_march2025_measuremetns_standardized_measurements_mk2.csv'

# Define the metrics to plot
metrics = ['circularity', 'eccentricity', 'solidity', 'extent', 'Minor Axis Length']

# Define which metrics are significant for each species
species_significance = {
    'LC': ['circularity', 'solidity', 'extent'],
    'LG': ['Minor Axis Length', 'eccentricity'],
    'LS': ['Minor Axis Length', 'extent', 'circularity'],
    'FV': ['circularity', 'eccentricity', 'solidity', 'extent', 'Minor Axis Length']
}

# Map species codes to full names
species_full_names = {
    'LC': 'Lottia conus',
    'LG': 'Lottia gigantea',
    'LS': 'Lottia strigatella',
    'FV': 'Fissurella volcano'
}

# Function to remove outliers using the IQR method
def remove_outliers(data):
    if len(data) == 0:
        return data
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data >= lower_bound) & (data <= upper_bound)]

# Create output folder on Desktop
desktop_path = os.path.expanduser('')
output_folder = os.path.join(desktop_path, 'morphometric_pdfs')
os.makedirs(output_folder, exist_ok=True)



#### Process LC, LG, LS species ####

for species_code in ['LC', 'LG', 'LS']:
    
    df = pd.read_csv(file_path_lc_lg_ls)
    
    # Extract orientation from the second part of the path
    df['orientation'] = df['image_path'].apply(lambda x: x.split('/')[1])
    # Extract region from the third part
    df['region'] = df['image_path'].apply(lambda x: x.split('/')[2].split('_')[0])
    # Extract species from the first part of the path
    df['species'] = df['image_path'].apply(lambda x: x.split('/')[0])
    
    # Calculate circularity
    df['circularity'] = (4 * np.pi * df['area']) / (df['perimeter'] ** 2)
    
    # Scale minor_axis_length by dividing by 100
    df['Minor Axis Length'] = df['minor_axis_length'] / 100
    
    # Map species names to shorthand labels
    species_mapping = {
        'lottia_gigantea': 'LG',
        'lottia_conus': 'LC',
        'lottia_strigatella': 'LS',
        'fissurella_volcano': 'FV'
    }
    df['species'] = df['species'].map(species_mapping)
    
    # Filter for current species
    df_species = df[df['species'] == species_code]
    
    if len(df_species) == 0:
        print(f"\nWARNING: No data found for species {species_code}")
        continue
    
    # Get significant metrics for this species
    significant_metrics = species_significance[species_code]
    
    # Print mean values
    print(f"\nMean values for {species_code} ({species_full_names[species_code]}):")
    print("=" * 60)
    
    for metric in metrics:
        data_north = remove_outliers(
            df_species[df_species['region'].fillna('').str.contains('north', case=False, na=False)][metric].dropna()
        )
        data_south = remove_outliers(
            df_species[df_species['region'].fillna('').str.contains('south', case=False, na=False)][metric].dropna()
        )
        
        if len(data_north) == 0 or len(data_south) == 0:
            print(f"{metric.replace('_', ' ').title()}: Insufficient data")
            continue
            
        mean_north = data_north.mean()
        mean_south = data_south.mean()
        sig_marker = "*" if metric in significant_metrics else ""
        print(f"{metric.replace('_', ' ').title()}: Northern = {mean_north:.3f}, Southern = {mean_south:.3f} {sig_marker}")
    


    #### Create Horizontal Boxplot ####

    
    plt.style.use('default')
    fig, ax = plt.subplots(figsize=(16, 10))  # Wider figure to accommodate legend on right
    
    # Define consistent colors for Northern and Southern
    northern_color = '#2E86AB'  # Deep blue
    southern_color = '#A23B72'  # Deep magenta/pink
    
    positions = []
    plot_data = []
    plot_colors = []
    y_ticks = []
    y_tick_labels = []
    
    offset = 0.2
    
    # Loop through each metric - REVERSED ORDER so Northern is on top
    for i, metric in enumerate(metrics):
        pos_northern = i + 1 + offset  # Northern on top (higher position)
        pos_southern = i + 1 - offset  # Southern on bottom (lower position)
        
        data_north = remove_outliers(
            df_species[df_species['region'].fillna('').str.contains('north', case=False, na=False)][metric].dropna()
        )
        data_south = remove_outliers(
            df_species[df_species['region'].fillna('').str.contains('south', case=False, na=False)][metric].dropna()
        )
        
        # Append Northern first (top), then Southern (bottom)
        positions.extend([pos_northern, pos_southern])
        plot_data.append(data_north)
        plot_data.append(data_south)
        plot_colors.extend([northern_color, southern_color])
        
        # Create label with asterisk if significant
        label = metric.replace('_', ' ').title()
        if metric in significant_metrics:
            label += " *"
        

y_tick_labels = []

offset = 0.2

for i, metric in enumerate(metrics):
    pos_northern = i + 1 + offset  # Northern on top
    pos_southern = i + 1 - offset  # Southern on bottom
    
    data_north = remove_outliers(
        df_FV[df_FV['region'].fillna('').str.contains('north', case=False, na=False)][metric].dropna()
    )        y_ticks.append(i + 1)
        y_tick_labels.append(label)
    
    # Create the horizontal boxplots
    bp = ax.boxplot(plot_data, positions=positions, widths=0.3, patch_artist=True,
                    boxprops=dict(linewidth=1.5, edgecolor='black'),
                    medianprops=dict(color='white', linewidth=2),
                    whiskerprops=dict(linewidth=1.5, color='black'),
                    capprops=dict(linewidth=1.5, color='black'),
                    flierprops=dict(marker='o', markerfacecolor='grey', markersize=4, alpha=0.6),
                    vert=False)
    
    for patch, color in zip(bp['boxes'], plot_colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.8)
    
    # Customize the plot - TEXT SIZE INCREASED BY 2X
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_tick_labels, fontsize=24)
    ax.set_xlabel("Standardized Value", fontsize=28)
    #ax.set_title(f"Morphometric Analysis: {species_full_names[species_code]} (Northern and Southern Clades)", 
                 #fontsize=32, fontweight='bold', pad=20)
    
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)
    ax.set_facecolor('#fafafa')
    
    # Create legend with "Clades" terminology
    legend_elements = [
        Patch(facecolor=northern_color, alpha=0.8, label='Northern Clade'),
        Patch(facecolor=southern_color, alpha=0.8, label='Southern Clade'),
        Patch(facecolor='white', alpha=0.0, label=''),
        Patch(facecolor='white', alpha=0.0, label='Statistical Significance:'),
        Patch(facecolor='white', alpha=0.0, label='* p < 0.05')
    ]
    
    # Position legend OUTSIDE the plot on the right, without squishing the graph
    ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1.05, 0.5), 
              fontsize=22,
              frameon=True, fancybox=True, shadow=True, 
              title='Legend', title_fontsize=24)
    
    # Don't use tight_layout - instead adjust manually to preserve graph size
    plt.subplots_adjust(right=0.75)  # Leave 25% space on right for legend
    
    output_path = os.path.join(output_folder, f'{species_code}_morphometric_analysis_horizontal.pdf')
    plt.savefig(output_path, dpi=600, bbox_inches='tight', format='pdf')
    print(f"\nPDF saved to: {output_path}")
    print("=" * 60)
    
    plt.show()
    plt.close()


#### Process FV species (different file structure) ####


df = pd.read_csv(file_path_fv)

# Extract orientation from the first part
df['orientation'] = df['image_path'].apply(lambda x: x.split('/')[0])
# Extract region from the second part
df['region'] = df['image_path'].apply(lambda x: x.split('/')[1].split('_')[0])
# Extract species from the filename
df['species'] = df['image_path'].apply(lambda x: x.split('/')[-1].split('_')[0].upper())

# Calculate circularity
df['circularity'] = (4 * np.pi * df['area']) / (df['perimeter'] ** 2)

# Scale minor_axis_length by dividing by 100
df['Minor Axis Length'] = df['minor_axis_length'] / 100

# Filter for species 'FV'
df_FV = df[df['species'] == 'FV']

significant_metrics = species_significance['FV']

print(f"\nMean values for FV ({species_full_names['FV']}):")
print("=" * 60)

for metric in metrics:
    data_north = remove_outliers(
        df_FV[df_FV['region'].fillna('').str.contains('north', case=False, na=False)][metric].dropna()
    )
    data_south = remove_outliers(
        df_FV[df_FV['region'].fillna('').str.contains('south', case=False, na=False)][metric].dropna()
    )
    mean_north = data_north.mean()
    mean_south = data_south.mean()
    print(f"{metric.replace('_', ' ').title()}: Northern = {mean_north:.3f}, Southern = {mean_south:.3f} *")

# Create Horizontal Boxplot for FV
plt.style.use('default')
fig, ax = plt.subplots(figsize=(16, 10))  # Wider figure to accommodate legend on right

northern_color = '#2E86AB'
southern_color = '#A23B72'

positions = []
plot_data = []
plot_colors = []
y_ticks = []
y_tick_labels = []

offset = 0.2

for i, metric in enumerate(metrics):
    pos_northern = i + 1 + offset  # Northern on top
    pos_southern = i + 1 - offset  # Southern on bottom
    
    data_north = remove_outliers(
        df_FV[df_FV['region'].fillna('').str.contains('north', case=False, na=False)][metric].dropna()
    )
    data_south = remove_outliers(
        df_FV[df_FV['region'].fillna('').str.contains('south', case=False, na=False)][metric].dropna()
    )
    
    # Northern first (top), then Southern (bottom)
    positions.extend([pos_northern, pos_southern])
    plot_data.append(data_north)
    plot_data.append(data_south)
    plot_colors.extend([northern_color, southern_color])
    
    label = metric.replace('_', ' ').title() + " *"
    
    y_ticks.append(i + 1)
    y_tick_labels.append(label)

bp = ax.boxplot(plot_data, positions=positions, widths=0.3, patch_artist=True,
                boxprops=dict(linewidth=1.5, edgecolor='black'),
                medianprops=dict(color='white', linewidth=2),
                whiskerprops=dict(linewidth=1.5, color='black'),
                capprops=dict(linewidth=1.5, color='black'),
                flierprops=dict(marker='o', markerfacecolor='grey', markersize=4, alpha=0.6),
                vert=False)

for patch, color in zip(bp['boxes'], plot_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.8)

ax.set_yticks(y_ticks)
ax.set_yticklabels(y_tick_labels, fontsize=24)
ax.set_xlabel("Standardized Value", fontsize=28)
#ax.set_title(f"Morphometric Analysis: {species_full_names['FV']} (Northern and Southern Clades)", 
             #fontsize=32, fontweight='bold', pad=20)

ax.grid(True, alpha=0.3, linestyle='--')
ax.set_axisbelow(True)
ax.set_facecolor('#fafafa')

legend_elements = [
    Patch(facecolor=northern_color, alpha=0.8, label='Northern Clade'),
    Patch(facecolor=southern_color, alpha=0.8, label='Southern Clade'),
    Patch(facecolor='white', alpha=0.0, label=''),
    Patch(facecolor='white', alpha=0.0, label='Statistical Significance:'),
    Patch(facecolor='white', alpha=0.0, label='* p < 0.05')
]

# Position legend OUTSIDE the plot on the right, without squishing the graph
ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1.05, 0.5),
          fontsize=22,
          frameon=True, fancybox=True, shadow=True, 
          title='Legend', title_fontsize=24)

# Don't use tight_layout - instead adjust manually to preserve graph size
plt.subplots_adjust(right=0.75)  # Leave 25% space on right for legend

output_path = os.path.join(output_folder, 'FV_morphometric_analysis_horizontal.pdf')
plt.savefig(output_path, dpi=600, bbox_inches='tight', format='pdf')
print(f"\nPDF saved to: {output_path}")
print("=" * 60)

plt.show()
plt.close()

print("\n\nAll morphometric visualizations complete!")
print(f"All 4 PDFs saved to: {output_folder}")

In [ ]:
###### Script to generate boxplots comparing F1-scores for all limpet species and even/full test models ####


##### Imports ####

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Common setup
normal_colour = 'skyblue'
mixed_colour = 'lightcoral'

def is_mixed(model_name):
    return "mixed" in model_name.lower()

def rename_label(label):
    label_no_underscore = label.replace('_', ' ').strip()
    tokens = label_no_underscore.split()
    mapping = {
        'fv': 'Fissurella volcano',
        'lc': 'Lottia conus',
        'lg': 'Lottia gigantea',
        'ls': 'Lottia strigettela'
    }
    if tokens and tokens[0].lower() in mapping:
        full_name = mapping[tokens[0].lower()]
        if len(tokens) > 1:
            orientation = tokens[1].capitalize()
            full_name += " " + orientation
        return full_name
    return label_no_underscore

def prepare_data(file):
    df = pd.read_csv(file)
    df.columns = df.columns.str.strip().str.lower()
    df['main folder'] = df['main folder'].str.replace('dorasl', 'dorsal', case=False)
    df['main folder'] = df['main folder'].str.lower()

    unique_combinations = df['main folder'].unique()
    sorted_combinations = sorted(unique_combinations, key=lambda x: (is_mixed(x), x))

    box_data, box_labels, box_colours = [], [], []

    for combination in sorted_combinations:
        subset = df[df['main folder'] == combination]['accuracy f1-score']
        if not subset.empty:
            box_data.append(subset.values)
            box_labels.append(rename_label(combination))
            box_colours.append(mixed_colour if is_mixed(combination) else normal_colour)

    return box_data, box_labels, box_colours, sorted_combinations

# Files
file1 = "100_run_aggregated_f1_scores_main_march25.csv"
file2 = "full_100_run_aggregated_f1_scores_main_march25.csv"

# Prepare data
box_data1, labels1, colours1, sorted_combinations1 = prepare_data(file1)
box_data2, labels2, colours2, sorted_combinations2 = prepare_data(file2)

# Plot setup
fig, axs = plt.subplots(2, 1, figsize=(20, 20))

# Plot first graph
box_plot1 = axs[0].boxplot(box_data1, patch_artist=True, widths=0.6)
for patch, colour in zip(box_plot1['boxes'], colours1):
    patch.set_facecolor(colour)
for i, median in enumerate(box_plot1['medians']):
    median.set_color('red' if not is_mixed(sorted_combinations1[i]) else 'blue')
    median.set_linewidth(2)
axs[0].set_title("F1-Score Distribution (Even test set)", fontsize=16)
#axs[0].set_xlabel("Main Folder (Species and Orientation)", fontsize=14)
axs[0].set_ylabel("Accuracy F1-Score", fontsize=14)
axs[0].set_xticklabels(labels1, rotation=45, fontsize=12)
axs[0].grid(axis='y', linestyle='--', alpha=0.6)

# Plot second graph
box_plot2 = axs[1].boxplot(box_data2, patch_artist=True, widths=0.6)
for patch, colour in zip(box_plot2['boxes'], colours2):
    patch.set_facecolor(colour)
for i, median in enumerate(box_plot2['medians']):
    median.set_color('red' if not is_mixed(sorted_combinations2[i]) else 'blue')
    median.set_linewidth(2)
axs[1].set_title("F1-Score Distribution (Full test set)", fontsize=16)
#axs[1].set_xlabel("Main Folder (Species and Orientation)", fontsize=14)
axs[1].set_ylabel("Accuracy F1-Score", fontsize=14)
axs[1].set_xticklabels(labels2, rotation=45, fontsize=12)
axs[1].grid(axis='y', linestyle='--', alpha=0.6)

# Shared legend
legend_elements = [
    Line2D([0], [0], marker='s', color='w', label='Normal Models', markerfacecolor=normal_colour, markersize=10),
    Line2D([0], [0], marker='s', color='w', label='Mixed Models', markerfacecolor=mixed_colour, markersize=10)
]

fig.legend(handles=legend_elements, loc='upper right', fontsize=12)
plt.tight_layout(rect=[0, 0, 0.95, 0.98])
plt.show()


In [ ]:
###### Script to generate combined mean shape plots for FV species (Krusk-Wallis) ######


##### Imports ####

import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import os

# Define colours for northern and southern shapes
northern_color = '#2E86AB'
southern_color = '#A23B72'

# Function to extract contours from a binary mask
def extract_contours(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not contours:
        return None
    largest_contour = max(contours, key=cv2.contourArea)
    largest_contour = largest_contour.squeeze().astype(np.float32)
    return largest_contour

# Function to resample a contour to a fixed number of points
def resample_contour(contour, num_points=100):
    if not np.allclose(contour[0], contour[-1]):
        contour = np.vstack([contour, contour[0]])
    distances = np.cumsum(np.sqrt(np.sum(np.diff(contour, axis=0)**2, axis=1)))
    distances = np.insert(distances, 0, 0)
    distances /= distances[-1]
    fx = interp1d(distances, contour[:, 0], kind='cubic')
    fy = interp1d(distances, contour[:, 1], kind='cubic')
    new_distances = np.linspace(0, 1, num_points)
    resampled_contour = np.vstack((fx(new_distances), fy(new_distances))).T
    return resampled_contour

# Function to normalise and centralise a contour
def normalise_and_centralise(contour):
    min_x, min_y = contour.min(axis=0)
    max_x, max_y = contour.max(axis=0)
    contour -= [min_x, min_y]
    scale = max(max_x - min_x, max_y - min_y)
    contour /= scale
    centroid = np.mean(contour, axis=0)
    contour -= centroid
    return contour

# Process all masks in a directory and compute the mean shape
def process_mask_directory_with_mean(dir_path, num_points=100):
    all_contours = []
    mask_paths = [os.path.join(dir_path, f) for f in os.listdir(dir_path)
                  if f.endswith(('.png', '.jpg', '.jpeg'))]
    for path in mask_paths:
        contour = extract_contours(path)
        if contour is not None:
            resampled_contour = resample_contour(contour, num_points)
            normalised_contour = normalise_and_centralise(resampled_contour)
            all_contours.append(normalised_contour)
    if not all_contours:
        return None
    return np.mean(np.array(all_contours), axis=0)

# Function to sample left and right points
def sample_points_by_y(contour, fractions=[0.75, 0.5, 0.25]):
    ys = contour[:, 1]
    y_top = np.min(ys)
    y_bottom = np.max(ys)
    sample_points_left = []
    sample_points_right = []
    
    for f in fractions:
        y_target = (1 - f) * y_bottom + f * y_top
        intersections = []
        n = len(contour)
        
        for i in range(n):
            p1 = contour[i]
            p2 = contour[(i+1) % n]
            if (p1[1] - y_target) * (p2[1] - y_target) <= 0:
                if p1[1] == p2[1]:
                    intersections.append(p1)
                else:
                    t = (y_target - p1[1]) / (p2[1] - p1[1])
                    x_intersect = p1[0] + t * (p2[0] - p1[0])
                    intersections.append(np.array([x_intersect, y_target]))

        if intersections:
            intersections = np.array(intersections)
            left_point = intersections[np.argmin(intersections[:, 0])]
            right_point = intersections[np.argmax(intersections[:, 0])]
        else:
            left_point = None
            right_point = None

        sample_points_left.append(left_point)
        sample_points_right.append(right_point)

    return sample_points_left, sample_points_right

# Function to plot results and save as PDF
def plot_combined_mean_shapes(combined_means, labels, sample_points_list=None,
                              output_path=None, dpi=600):
    plt.figure(figsize=(8, 8))

    # Plot each shape
    for i, (combined_mean, label) in enumerate(zip(combined_means, labels)):
        if combined_mean is not None:
            # Choose colour based on label (north vs south)
            if 'north' in label.lower():
                colour = northern_color
            elif 'south' in label.lower():
                colour = southern_color
            else:
                colour = None

            if colour is not None:
                plt.plot(combined_mean[:, 0], combined_mean[:, 1],
                         linewidth=2, label=label, color=colour)
            else:
                plt.plot(combined_mean[:, 0], combined_mean[:, 1],
                         linewidth=2, label=label)

            if sample_points_list is not None and sample_points_list[i] is not None:
                left_points, right_points = sample_points_list[i]
                
                if left_points is not None:
                    left_points_arr = np.array([p for p in left_points if p is not None])
                    if len(left_points_arr) > 0:
                        plt.plot(left_points_arr[:, 0], left_points_arr[:, 1],
                                 'o', markersize=3, color='blue')

                if right_points is not None:
                    right_points_arr = np.array([p for p in right_points if p is not None])
                    if len(right_points_arr) > 0:
                        plt.plot(right_points_arr[:, 0], right_points_arr[:, 1],
                                 'o', markersize=3, color='red')

    # Annotate points (across both shapes)
    if sample_points_list is not None and len(sample_points_list) == 2:
        left_labels = ['A', 'B', 'C']
        right_labels = ['D', 'E', 'F']

        for j in range(3):  # Three fractions (75%, 50%, 25%)
            left_pts_0, right_pts_0 = sample_points_list[0]
            left_pts_1, right_pts_1 = sample_points_list[1]

            left_pt = left_pts_0[j] if left_pts_0[j] is not None else left_pts_1[j]
            right_pt = right_pts_0[j] if right_pts_0[j] is not None else right_pts_1[j]

            if left_pt is not None:
                plt.text(left_pt[0] - 0.02, left_pt[1], left_labels[j], fontsize=12,
                         verticalalignment='center', horizontalalignment='right', color='blue')
            if right_pt is not None:
                plt.text(right_pt[0] + 0.02, right_pt[1], right_labels[j], fontsize=12,
                         verticalalignment='center', horizontalalignment='left', color='red')

    plt.gca().invert_yaxis()
    plt.axis('equal')
    plt.xticks([])
    plt.yticks([])
    plt.title("Combined Mean Shapes with Sample Points")
    plt.legend()

    # Save as PDF at the requested resolution
    if output_path is not None:
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight')

    plt.show()

# Directories for FV species - need to point to hole masks
dir_fv_d_s = "/dorsal/south_dorsal/"
dir_fv_v_s = "/ventral/south_ventral/"
dir_fv_d_n = "/dorsal/north_dorsal/"
dir_fv_v_n = "/ventral/north_ventral/"

combined_mean_s = process_mask_directory_with_mean(dir_fv_d_s)
combined_mean_n = process_mask_directory_with_mean(dir_fv_d_n)

# Sample points on each shape
fractions = [0.75, 0.5, 0.25]
sample_points_s = sample_points_by_y(combined_mean_s, fractions)
sample_points_n = sample_points_by_y(combined_mean_n, fractions)

# Plot the results:
#   - North first in the list so it appears at the top of the legend
#   - Save as a 600 dpi PDF
plot_combined_mean_shapes(
    [combined_mean_n, combined_mean_s],
    ["Mean Northern Shape", "Mean Southern Shape"],
    sample_points_list=[sample_points_n, sample_points_s],
    output_path="fv_north_south_mean_shapes.pdf",
    dpi=600
)
